url : https://www.kaggle.com/datasets/jessemostipak/hotel-booking-demand                   
download : https://www.kaggle.com/datasets/jessemostipak/hotel-booking-demand/download?datasetVersionNumber=1

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [111]:
data = pd.read_csv('/kaggle/input/hotel-booking-demand/hotel_bookings.csv')

In [131]:
data

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,Year,month,Day,C__ABW,C__AGO,C__AIA,C__ALB,C__AND,C__ARE,C__ARG,C__ARM,C__ASM,C__ATA,C__ATF,C__AUS,C__AUT,C__AZE,C__BDI,C__BEL,C__BEN,C__BFA,C__BGD,C__BGR,C__BHR,C__BHS,C__BIH,C__BLR,C__BOL,C__BRA,C__BRB,C__BWA,C__CAF,C__CHE,C__CHL,C__CHN,C__CIV,C__CMR,C__CN,C__COL,C__COM,C__CPV,C__CRI,C__CUB,C__CYM,C__CYP,C__CZE,C__DEU,C__DJI,C__DMA,C__DNK,C__DOM,C__DZA,C__ECU,C__EGY,C__ESP,C__EST,C__ETH,C__FIN,C__FJI,C__FRA,C__FRO,C__GAB,C__GBR,C__GEO,C__GGY,C__GHA,C__GIB,C__GLP,C__GNB,C__GRC,C__GTM,C__GUY,C__HKG,C__HND,C__HRV,C__HUN,C__IDN,C__IMN,C__IND,C__IRL,C__IRN,C__IRQ,C__ISL,C__ISR,C__ITA,C__JAM,C__JEY,C__JOR,C__JPN,C__KAZ,C__KEN,C__KHM,C__KIR,C__KNA,C__KOR,C__KWT,C__LAO,C__LBN,C__LBY,C__LCA,C__LIE,C__LKA,C__LTU,C__LUX,C__LVA,C__MAC,C__MAR,C__MCO,C__MDG,C__MDV,C__MEX,C__MKD,C__MLI,C__MLT,C__MMR,C__MNE,C__MOZ,C__MRT,C__MUS,C__MWI,C__MYS,C__MYT,C__NAM,C__NCL,C__NGA,C__NIC,C__NLD,C__NOR,C__NPL,C__NZL,C__Nothing,C__OMN,C__PAK,C__PAN,C__PER,C__PHL,C__PLW,C__POL,C__PRI,C__PRT,C__PRY,C__PYF,C__QAT,C__ROU,C__RUS,C__RWA,C__SAU,C__SDN,C__SEN,C__SGP,C__SLE,C__SLV,C__SMR,C__SRB,C__STP,C__SUR,C__SVK,C__SVN,C__SWE,C__SYC,C__SYR,C__TGO,C__THA,C__TJK,C__TMP,C__TUN,C__TUR,C__TWN,C__TZA,C__UGA,C__UKR,C__UMI,C__URY,C__USA,C__UZB,C__VEN,C__VGB,C__VNM,C__ZAF,C__ZMB,C__ZWE,M__Aviation,M__Complementary,M__Corporate,M__Direct,M__Groups,M__Offline TA/TO,M__Online TA,M__Undefined,RES__Canceled,RES__Check-Out,RES__No-Show,DIST__Corporate,DIST__Direct,DIST__GDS,DIST__TA/TO,DIST__Undefined,DEPOST__No Deposit,DEPOST__Non Refund,DEPOST__Refundable
0,0,0,342,2015,27,1,0,0,2,0.0,0,0,0,0,0,2,2,3,86.693382,0,2,0.00,0,0,2015.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0,0,737,2015,27,1,0,0,2,0.0,0,0,0,0,0,2,2,4,86.693382,0,2,0.00,0,0,2015.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0,0,7,2015,27,1,0,1,1,0.0,0,0,0,0,0,0,2,0,86.693382,0,2,75.00,0,0,2015.0,7.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.

In [128]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Columns: 224 entries, hotel to DEPOST__Refundable
dtypes: float64(203), int64(21)
memory usage: 204.0 MB


In [114]:
data.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

# PreProcessing

## Missing Values

In [115]:
data = data.drop('company',axis=1)

In [116]:
data['country'] = data['country'].replace(np.nan,'Nothing')

In [117]:
data['children'] = data['children'].fillna(data['children'].mean())

In [118]:
data['agent']= data['agent'].fillna(data['agent'].mean())

### Encoding Cateforical Values

In [127]:
{col : data[col].unique() for col in data.columns if data[col].dtypes == 'object'}

{}

In [120]:
data = data.drop('arrival_date_month',axis=1)

In [121]:
data['Year'] = data['reservation_status_date'].apply(lambda x: x[0:4]).astype(np.float)
data['month'] = data['reservation_status_date'].apply(lambda x: x[5:7]).astype(np.float)
data['Day'] = data['reservation_status_date'].apply(lambda x: x[8:11]).astype(np.float)
data = data.drop('reservation_status_date',axis=1)

/tmp/ipykernel_32/1871723653.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data['Year'] = data['reservation_status_date'].apply(lambda x: x[0:4]).astype(np.float)
/tmp/ipykernel_32/1871723653.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data['month'] = data['reservation_status_date'].apply(lambda x: x[5:7]).astype(np.float)
/

In [122]:
{col : data[col].unique() for col in data.columns if data[col].dtypes == 'object'}

{'hotel': array(['Resort Hotel', 'City Hotel'], dtype=object),
 'meal': array(['BB', 'FB', 'HB', 'SC', 'Undefined'], dtype=object),
 'country': array(['PRT', 'GBR', 'USA', 'ESP', 'IRL', 'FRA', 'Nothing', 'ROU', 'NOR',
        'OMN', 'ARG', 'POL', 'DEU', 'BEL', 'CHE', 'CN', 'GRC', 'ITA',
        'NLD', 'DNK', 'RUS', 'SWE', 'AUS', 'EST', 'CZE', 'BRA', 'FIN',
        'MOZ', 'BWA', 'LUX', 'SVN', 'ALB', 'IND', 'CHN', 'MEX', 'MAR',
        'UKR', 'SMR', 'LVA', 'PRI', 'SRB', 'CHL', 'AUT', 'BLR', 'LTU',
        'TUR', 'ZAF', 'AGO', 'ISR', 'CYM', 'ZMB', 'CPV', 'ZWE', 'DZA',
        'KOR', 'CRI', 'HUN', 'ARE', 'TUN', 'JAM', 'HRV', 'HKG', 'IRN',
        'GEO', 'AND', 'GIB', 'URY', 'JEY', 'CAF', 'CYP', 'COL', 'GGY',
        'KWT', 'NGA', 'MDV', 'VEN', 'SVK', 'FJI', 'KAZ', 'PAK', 'IDN',
        'LBN', 'PHL', 'SEN', 'SYC', 'AZE', 'BHR', 'NZL', 'THA', 'DOM',
        'MKD', 'MYS', 'ARM', 'JPN', 'LKA', 'CUB', 'CMR', 'BIH', 'MUS',
        'COM', 'SUR', 'UGA', 'BGR', 'CIV', 'JOR', 'SYR', 'SGP', 'BDI',
  

In [123]:
binary_fet = ['hotel']
ordinal_fet = ['meal','reserved_room_type','assigned_room_type','customer_type']
nominal_fet = ['country','market_segment','reservation_status','distribution_channel','deposit_type']
nominal_prefix = ['C_' , 'M_' , 'RES_' , 'DIST_','DEPOST_']

In [124]:
data['hotel'] = data['hotel'].apply(lambda x: 1 if x == 'City Hotel' else  0 )

In [125]:
for ordinal in ordinal_fet:
    ordined = sorted(list(data[ordinal].unique()))
    data[ordinal] = data[ordinal].apply(lambda x : ordined.index(x))

In [126]:
for one_hot , prefixs in zip(nominal_fet,nominal_prefix):
    dummie = pd.get_dummies(data[one_hot],prefix=prefixs).astype(np.float)
    data = pd.concat([data,dummie],axis=1)
    data = data.drop(one_hot,axis=1)

/tmp/ipykernel_32/38820349.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dummie = pd.get_dummies(data[one_hot],prefix=prefixs).astype(np.float)
/tmp/ipykernel_32/38820349.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dummie = pd.get_dummies(data[one_hot],prefix=prefixs).astype(np.float)
/tmp/ipykernel_32/38820349.py:2: Depreca

# Selecting , Scaling and Spliting

In [132]:
y = data['hotel']
x = data.drop('hotel',axis=1)

In [142]:
scaler  = StandardScaler()
x = pd.DataFrame(scaler.fit_transform(x),columns=x.columns)

In [143]:
x_train , x_test , y_train , y_test = train_test_split(x,y,train_size=0.7,random_state=7)

In [145]:
#y.sum() / len(y)
#x.shape

(119390, 223)

# Training Our Child 

In [157]:
inputs = tf.keras.Input(shape=(223,))
x = tf.keras.layers.Dense(256,activation='relu')(inputs)
x = tf.keras.layers.Dense(256,activation='relu')(x)
outputs = tf.keras.layers.Dense(1,activation='sigmoid')(x)

model = tf.keras.Model(inputs = inputs , outputs = outputs)

model.compile(
    optimizer = 'adam',
    loss  = 'binary_crossentropy',
    metrics = ['accuracy',
              tf.keras.metrics.AUC(name='auc')]
)
epochs = 8

hist = model.fit(
    x_train,
    y_train,
    validation_split = 0.2,
    batch_size = 64,
    epochs = epochs,
    callbacks=tf.keras.callbacks.ReduceLROnPlateau(),
    verbose = 0
)

Epoch 1/8
1045/1045 [==============================] - 5s 4ms/step - loss: 0.1751 - accuracy: 0.9363 - auc: 0.9795 - val_loss: 0.1202 - val_accuracy: 0.9542 - val_auc: 0.9897 - lr: 0.0010
Epoch 2/8
1045/1045 [==============================] - 4s 4ms/step - loss: 0.1022 - accuracy: 0.9623 - auc: 0.9922 - val_loss: 0.0994 - val_accuracy: 0.9640 - val_auc: 0.9933 - lr: 0.0010
Epoch 3/8
1045/1045 [==============================] - 4s 4ms/step - loss: 0.0823 - accuracy: 0.9689 - auc: 0.9948 - val_loss: 0.0888 - val_accuracy: 0.9667 - val_auc: 0.9945 - lr: 0.0010
Epoch 4/8
1045/1045 [==============================] - 5s 5ms/step - loss: 0.0672 - accuracy: 0.9742 - auc: 0.9966 - val_loss: 0.0824 - val_accuracy: 0.9725 - val_auc: 0.9959 - lr: 0.0010
Epoch 5/8
1045/1045 [==============================] - 4s 4ms/step - loss: 0.0556 - accuracy: 0.9789 - auc: 0.9975 - val_loss: 0.0832 - val_accuracy: 0.9740 - val_auc: 0.9955 - lr: 0.0010
Epoch 6/8
1045/1045 [==============================] - 4s 4m

# Result

In [158]:
fig = px.line(
    hist.history,
    y = ['loss' , 'val_loss'],
    labels = {'x' : 'Epochs' , 'y' : 'Loss'},
    title = 'training and validation loss'
)
fig.show()

In [159]:
model.evaluate(x_test , y_test)

1120/1120 [==============================] - 2s 2ms/step - loss: 0.0718 - accuracy: 0.9759 - auc: 0.9958


[0.0718013122677803, 0.9759331941604614, 0.9958083033561707]

In [ ]:
np.argmin(hist.history['val_loss']) + 1 

### Tanko you so much for watching 
#### **i hope you enjoy** :)